In [1]:
from mxnet import nd, gluon
from mxnet.gluon.model_zoo import vision as models
from sklearn.metrics import accuracy_score
from mxnet import autograd

import mxnet as mx
import numpy as np
import pickle
import os

In [2]:
data_dir = '/media/tx-deepocean/Data/dataset/cifar10/cifar-10-batches-py'

train_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file = 'test_batch'

batch_size = 500
ctx = mx.cpu()

In [3]:
train_data = []
train_label = []

for train_file in train_files:
    with open(os.path.join(data_dir, train_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        train_data.append(np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32)))
        train_label.extend(tmp[b'labels'])
train_data = np.concatenate(train_data, axis=0)

val_data = []
val_label = []
with open(os.path.join(data_dir, test_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        val_data = np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32))
        val_label = tmp[b'labels']

In [4]:
train_dataset = gluon.data.dataset.ArrayDataset(nd.array(train_data, ctx=ctx), nd.array(train_label, ctx=ctx))
train_loader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)

val_dataset = gluon.data.dataset.ArrayDataset(nd.array(val_data, ctx=ctx), nd.array(val_label, ctx=ctx))
val_loader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)

In [5]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [34]:
resnet18 = models.resnet18_v2(classes=10)
resnet18.initialize(force_reinit=True, ctx=[mx.gpu(0)], init=mx.init.Xavier())
trainer = gluon.Trainer(resnet18.collect_params(), 'sgd', {'learning_rate': 0.001})

In [35]:
for epoch in range(30):
    print('Epoch {}'.format(epoch + 1))
    train_loss = mx.nd.zeros(1, ctx=mx.gpu(0))
    training_samples = 0
    train_preds = None
    for X, y in train_loader:
        X = X.copyto(mx.gpu(0))
        y = y.copyto(mx.gpu(0))
        pred = None
        with autograd.record():
            output = resnet18(X)
            l = loss(output, y)
            output = nd.softmax(output)
            pred = np.argmax(output.asnumpy(), axis=1)
        l.backward()
        trainer.step(X.shape[0])
        if train_preds is None:
            train_preds = pred
        else:
            train_preds = np.concatenate((train_preds, pred))
        
        train_loss += l.sum()
        training_samples += X.shape[0]
    
    train_acc = accuracy_score(train_preds, train_label)
    
    train_loss = train_loss.asscalar()/training_samples
    
    val_loss = mx.nd.zeros(1, ctx=mx.gpu(0))
    val_preds = None
    val_samples = 0
    for X, y in val_loader:
        X = X.copyto(mx.gpu(0))
        y = y.copyto(mx.gpu(0))
        output = resnet18(X)
        l = loss(output, y)
        output = nd.softmax(output)
        val_loss += l.sum()
        pred = np.argmax(output.asnumpy(), axis=1)
        
        if val_preds is None:
            val_preds = pred
        else:
            val_preds = np.concatenate((val_preds, pred))
        
        val_samples += X.shape[0]
    val_acc = accuracy_score(val_preds, val_label)
    val_loss = val_loss.asscalar() / val_samples
    
    print('Train loss: {:.3f} acc: {:.3f}'.format(train_loss, train_acc))
    print('Val loss: {:.3f} acc: {:.3f}'.format(val_loss, val_acc))
        
    

Epoch 1
Train loss: 2.290 acc: 0.190
Val loss: 2.106 acc: 0.253
Epoch 2
Train loss: 1.988 acc: 0.293
Val loss: 1.919 acc: 0.317
Epoch 3
Train loss: 1.828 acc: 0.348
Val loss: 1.818 acc: 0.348
Epoch 4
Train loss: 1.720 acc: 0.388
Val loss: 1.752 acc: 0.370
Epoch 5
Train loss: 1.640 acc: 0.415
Val loss: 1.702 acc: 0.389
Epoch 6
Train loss: 1.576 acc: 0.440
Val loss: 1.665 acc: 0.401
Epoch 7
Train loss: 1.523 acc: 0.460
Val loss: 1.636 acc: 0.412
Epoch 8
Train loss: 1.477 acc: 0.477
Val loss: 1.612 acc: 0.420
Epoch 9
Train loss: 1.437 acc: 0.494
Val loss: 1.592 acc: 0.428
Epoch 10
Train loss: 1.400 acc: 0.509
Val loss: 1.575 acc: 0.434
Epoch 11
Train loss: 1.367 acc: 0.523
Val loss: 1.560 acc: 0.439
Epoch 12
Train loss: 1.336 acc: 0.534
Val loss: 1.548 acc: 0.443
Epoch 13
Train loss: 1.307 acc: 0.545
Val loss: 1.536 acc: 0.447
Epoch 14
Train loss: 1.280 acc: 0.556
Val loss: 1.526 acc: 0.450
Epoch 15
Train loss: 1.254 acc: 0.567
Val loss: 1.518 acc: 0.453
Epoch 16
Train loss: 1.229 acc: 0.